In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os

In [ ]:
! rm *.hdf
! rm *.gz
! rm *.tar
! rm *.tar*

In [ ]:
com0='http://orca.science.oregonstate.edu/data/1x2/8day/chl.modis.r2022/hdf/chl.m.20'
yr=['{0:02d}'.format(i) for i in np.arange(3,23)]
year=np.arange(2003,2023,1)
dayinmon=[31,28,31,30,31,30,31,31,30,31,30,31]
cday=np.cumsum(dayinmon)

In [ ]:
# download data and generate HDF files
def get_data(yr0):
    os.system('wget '+com0+yr0+'.tar >/dev/null 2>&1')
    os.system('tar -xvf chl.m.20'+yr0+'.tar >/dev/null 2>&1')
    os.system('rm chl.m.20'+yr0+'.tar')
    os.system('gunzip *.gz')

In [ ]:
# read in HDF file and save GoA data in netCDF format
def gen_netcdf(year):
    ! ls *.hdf > fn.txt
    fn=np.genfromtxt('fn.txt',dtype='U16')
    N=np.size(fn)
    #print(N)
    ds0=xr.open_dataset('../npp_CAFE-MODIS_8day_GOA.nc')
    time=ds0.time.sel(time=slice(str(year)+'-01-01',str(year+1)+'-01-01')).to_numpy()
    #print(time.shape)
    tmp=np.zeros((N,120,240))
    for n in range(N):
        ds=xr.open_dataset(fn[n], engine="netcdf4")
        x=np.linspace(-180,180,2160)
        y=np.linspace(-90,90,1080)
        chl=ds.chl.to_numpy()
        chl[chl==-9999]=np.nan
        da=xr.DataArray(data=chl[::-1,:],name='chl',dims=['lat','lon'],
               coords={'lat':y,'lon':x})
        da2=da.sel(lat=slice(42,62)).sel(lon=slice(-160,-120))
        x0=da2.lon.to_numpy()
        y0=da2.lat.to_numpy()
        tmp[n,:,:]=da2.to_numpy()
    #
    da=xr.DataArray(data=tmp,name='chl',dims=['time','lat','lon'],\
                   coords={'time':time[:N],'lat':y0,'lon':x0})
    ds=da.to_dataset()
    ! rm *.hdf
    return ds

In [ ]:
Nyr = np.size(yr)
for n in range(Nyr):
    get_data(yr[n])
    ds=gen_netcdf(year[n])
    ds.to_netcdf('npp_Chl-MODIS_8day_GOA_'+str(year[n])+'.nc')

In [ ]:
ds=xr.open_mfdataset('npp_Chl-MODIS_8day_GOA_*.nc')

In [ ]:
ds.to_netcdf('chl_MODIS_8day_GOA.nc')